# Draw Bounding Boxes

This Jupyter notebook should be opened using the following commands:
~~~bash
    workon jupyter
    jupyter notebook
~~~

In [1]:
import pandas as pd
import cv2
import argparse
import sys

In [2]:
# Uncomment the following line to test code within a Jupyter notebook
sys.argv = ['draw_bounding_boxes.py', 'data/ants/VID_20190810_120737', 'VID_20190810_120737.mp4']

parser = argparse.ArgumentParser(description='Draws bounding boxes on video frames.' \
                                 'Requires bounding boxes and frames for moving objects generated' \
                                 'by motion_detector.py.' )

parser.add_argument('data_dir',
                    type=str,
                    help='data directory (without trailing /) \
                    Example: data/ants/VID_20190810_120737')  

parser.add_argument('video_file',
                    type=str,
                    help='video file name \
                    Example: VID_20190810_120737.mp4')

parser.add_argument('--delay_ms',
                    default=33,
                    help='delay between frames in milliseconds (default=33)' \
                    'This serves to slow down sequential display of frames.')
                    
args = parser.parse_args()

In [3]:
# Load bounding_boxes.csv into a data frame.
fn = '{}/{}_bounding_boxes.csv'.format(args.data_dir, args.video_file.split('.')[0])
df = pd.read_csv(fn)

# Get dimensions of first frame, we assume that all frames have the same size.
path = df['filename'][0]
print(path)
img = cv2.imread(path)
frame_height, frame_width, _ = img.shape
print('frame_height: {}, frame_width: {}'.format(frame_height, frame_width))

# Convert coordinates from proportions to pixels
df['xtl'] = df['xtl'].apply(lambda x: int(x * frame_width))
df['ytl'] = df['ytl'].apply(lambda x: int(x * frame_height))
df['xbr'] = df['xbr'].apply(lambda x: int(x * frame_width))
df['ybr'] = df['ybr'].apply(lambda x: int(x * frame_height))

# Group by frame
grouped = df.groupby('filename')

# Iterate over frames
for name, df_group in grouped:
    img = cv2.imread(name, cv2.IMREAD_UNCHANGED)
    
    # Draw bounding boxes on each frame
    for index, row in df_group.iterrows():
        filename, xtl, ytl, xbr, ybr = row
        cv2.rectangle(img, (xtl, ytl), (xbr, ybr), (0, 255, 0), 3)

    # Show image with bounding boxes
    cv2.imshow(args.data_dir, img)
    cv2.waitKey(args.delay_ms)

cv2.destroyAllWindows()

data/ants/VID_20190810_120737/frames/VID_20190810_120737f000001.jpg
frame_height: 1080, frame_width: 1920


In [4]:
# Uncomment the following line to create a pure python script called draw_bounding_boxes.py
#!jupyter nbconvert --to=script 'draw_bounding_boxes.ipynb'